In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir('/content/drive/My Drive/')

In [ ]:
import keras
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,  Dropout
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [ ]:
dataset = pd.read_csv('all_comments.csv')
dataset = dataset.drop('Unnamed: 0', 1)
dataset.head()

,Review,Rating
0,Yerli ürün üretmemiz gerçekten çok iyi bir şey...,1
1,Monster teknik servisi çok kaliteli gösterdikl...,1
2,@gfbbahadir Şirkette 15 mimara 12 inşaat mühen...,1
3,@gfbbahadir Şirkette 15 mimara 12 inşaat mühen...,1
4,Arkadaşlar kıyaslama yaptığınızda aynı özellik...,1


In [ ]:
dataset.Rating.unique()

array([1, 0, 2])

In [ ]:
# get all labels and reviews as a list
dataset = dataset.sample(frac=1).reset_index(drop=True)
target = dataset['Rating'].values.tolist()
data = dataset['Review'].values.tolist()

In [ ]:
seperation = int(len(data) * 0.80)
x_train, x_test = data[:seperation], data[seperation:]
y_train, y_test = target[:seperation], target[seperation:]

In [ ]:
dataset.shape

(265299, 2)

In [ ]:
from gensim.models import Word2Vec
Embedding_dimensions = 100

# Creating Word2Vec training dataset.
Word2vec_train_data = list(map(lambda x: x.split(), x_train))

In [ ]:
# Defining the model and training it.
word2vec_model = Word2Vec(Word2vec_train_data,
                 size=Embedding_dimensions,
                 workers=8,
                 min_count=5)



In [ ]:
# Defining the model input length.
input_length = 59

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_length = 60000

tokenizer = Tokenizer(num_words=vocab_length)
tokenizer.fit_on_texts(data)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)

Tokenizer vocab length: 60000


In [ ]:
# saving tokenizer
import pickle

with open('/content/drive/My Drive/Red_and_Grey/bilstm_word2vec.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
X_train = pad_sequences(tokenizer.texts_to_sequences(x_train), maxlen=input_length)
X_test  = pad_sequences(tokenizer.texts_to_sequences(x_test) , maxlen=input_length)

print("X_train.shape:", X_train.shape)
print("X_test.shape :", X_test.shape)

X_train.shape: (212239, 59)
X_test.shape : (53060, 59)


In [ ]:
embedding_matrix = np.zeros((vocab_length, Embedding_dimensions))

for word, token in tokenizer.word_index.items():
    if word2vec_model.wv.__contains__(word):
        print(token)
        embedding_matrix[token] = word2vec_model.wv.__getitem__(word)

print("Embedding Matrix Shape:", embedding_matrix.shape)

In [ ]:
print("Embedding Matrix Shape:", embedding_matrix.shape)

Embedding Matrix Shape: (60000, 100)


In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False,
                                name='embedding_layer'))



In [ ]:
# 3-layered LSTM
model.add(LSTM(units=16, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=8, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=4, return_sequences=False))
model.add(Dropout(0.2))
# Dense layer: aka fully connected layer. Consists of one neuron.
model.add(Dense(3, activation='softmax'))#sigmoid

In [ ]:
from tensorflow.keras.optimizers import Adam
# Adam optimizer
optimizer = Adam(lr=1e-3)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# try using different optimizers and different optimizer configs
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
model.compile(loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])

In [ ]:
x_train_pad = np.array(X_train)
y_train = np.array(y_train)
x_test_pad = np.array(X_test)
y_test = np.array(y_test)

x_train_pad = np.array(x_train_pad)
y_train = np.array(y_train)
x_test_pad = np.array(x_test_pad)
y_test = np.array(y_test)

In [ ]:
# Reserve 10,000 samples for validation
x_val = x_train_pad[-10000:]
y_val = y_train[-10000:]
x_train = x_train_pad[:-10000]
y_train = y_train[:-10000]

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)


In [ ]:
y_val = to_categorical(y_val, 3)

In [ ]:
# epoch -> how many times we are going to train our data.
# batch_size -> feeding size
model.fit(x_train, y_train, epochs=15, batch_size=256,validation_data=(x_val, y_val),callbacks=[callback])

Epoch 1/15
790/790 [==============================] - 82s 97ms/step - loss: 0.3289 - accuracy: 0.8926 - val_loss: 0.2418 - val_accuracy: 0.9009
Epoch 2/15
790/790 [==============================] - 77s 97ms/step - loss: 0.2461 - accuracy: 0.9080 - val_loss: 0.2135 - val_accuracy: 0.9159
Epoch 3/15
790/790 [==============================] - 76s 96ms/step - loss: 0.2243 - accuracy: 0.9156 - val_loss: 0.2054 - val_accuracy: 0.9214
Epoch 4/15
790/790 [==============================] - 77s 97ms/step - loss: 0.2123 - accuracy: 0.9195 - val_loss: 0.1892 - val_accuracy: 0.9260
Epoch 5/15
790/790 [==============================] - 76s 97ms/step - loss: 0.2046 - accuracy: 0.9226 - val_loss: 0.1815 - val_accuracy: 0.9310
Epoch 6/15
790/790 [==============================] - 77s 97ms/step - loss: 0.1984 - accuracy: 0.9249 - val_loss: 0.1801 - val_accuracy: 0.9319
Epoch 7/15
790/790 [==============================] - 77s 97ms/step - loss: 0.1940 - accuracy: 0.9257 - val_loss: 0.1768 - val_accuracy:

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test_pad, y_test)
print("test loss, test acc:", results)

Evaluate on test data
1659/1659 [==============================] - 22s 13ms/step - loss: 0.1717 - accuracy: 0.9365
test loss, test acc: [0.1717449128627777, 0.9365058541297913]


In [ ]:
#lstm model #2
from keras.layers import Dense, Embedding, LSTM,  Dropout, CuDNNLSTM, Bidirectional, LeakyReLU


# 3-layered LSTM
model = Sequential()

embedding_size = 50 # vector with 50 size for every word

# Add embedding layer to our model.
# embedding matris size = num_words * embedding_size -> 10.000 * 50
model.add(Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False,
                                name='embedding_layer'))
model.add(Bidirectional(LSTM(units=16, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=8, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=4, return_sequences=False)))
model.add(Dropout(0.2))
# Dense layer: aka fully connected layer. Consists of one neuron.
model.add(Dense(3, activation='softmax'))#sigmoid

from tensorflow.keras.optimizers import Adam
# Adam optimizer
optimizer = Adam(learning_rate=1e-3)

# try using different optimizers and different optimizer configs
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
model.compile(loss='categorical_crossentropy',
optimizer=optimizer,
metrics=['accuracy'])




# epoch -> how many times we are going to train our data.
# batch_size -> feeding size
model.fit(x_train, y_train, epochs=15, batch_size=256,validation_data=(x_val, y_val),callbacks=[callback])


print("Evaluate on test data")
results = model.evaluate(x_test_pad, y_test)
print("test loss, test acc:", results)


Epoch 1/15
790/790 [==============================] - 150s 179ms/step - loss: 0.2963 - accuracy: 0.8950 - val_loss: 0.1999 - val_accuracy: 0.9246
Epoch 2/15
790/790 [==============================] - 140s 178ms/step - loss: 0.2130 - accuracy: 0.9190 - val_loss: 0.1824 - val_accuracy: 0.9318
Epoch 3/15
790/790 [==============================] - 139s 176ms/step - loss: 0.1946 - accuracy: 0.9267 - val_loss: 0.1757 - val_accuracy: 0.9348
Epoch 4/15
790/790 [==============================] - 140s 177ms/step - loss: 0.1844 - accuracy: 0.9306 - val_loss: 0.1696 - val_accuracy: 0.9362
Epoch 5/15
790/790 [==============================] - 140s 177ms/step - loss: 0.1776 - accuracy: 0.9336 - val_loss: 0.1705 - val_accuracy: 0.9372
Epoch 6/15
790/790 [==============================] - 140s 177ms/step - loss: 0.1722 - accuracy: 0.9356 - val_loss: 0.1640 - val_accuracy: 0.9372
Epoch 7/15
790/790 [==============================] - 140s 177ms/step - loss: 0.1679 - accuracy: 0.9370 - val_loss: 0.1629 -

In [ ]:
model.save("/content/drive/My Drive/Red_and_Grey/bilstm_word2vec_model.h5")


In [ ]:
https://www.kaggle.com/stoicstatic/twitter-sentiment-analysis-using-word2vec-bilstm

SyntaxError: ignored

BİLSTM WITH WORD2VEC


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight



df, df_val = train_test_split(dataset, test_size=0.20, random_state=42)
labels = df[['Review', 'Rating']]
classes = sorted(labels.Rating.unique())
print(classes)

class_weights = class_weight.compute_class_weight('balanced', np.unique(sorted(labels.Rating)), labels.Rating)
print(class_weights)

class_weight_dict = dict(enumerate(class_weights))
print(class_weight_dict)


[0, 1, 2]
[3.98907997 0.3793064  8.85546794]
{0: 3.9890799736866835, 1: 0.37930640073631255, 2: 8.855467935077398}


In [ ]:
from keras.layers import Dense, Embedding, LSTM,  Dropout, CuDNNLSTM, Bidirectional, LeakyReLU

x_train_pad = np.array(X_train)
y_train = np.array(y_train)
x_test_pad = np.array(X_test)
y_test = np.array(y_test)

# 3-layered LSTM
model = Sequential()

embedding_size = 50 # vector with 50 size for every word

model.add(Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False,
                                name='embedding_layer'))

# 3-layered LSTM
model.add(LSTM(units=16, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=8, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=4, return_sequences=False))
model.add(Dropout(0.2))
# Dense layer: aka fully connected layer. Consists of one neuron.
model.add(Dense(3, activation='softmax'))#sigmoid


from tensorflow.keras.optimizers import Adam
# Adam optimizer
optimizer = Adam(lr=1e-3, decay=1e-6)

# try using different optimizers and different optimizer configs
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model.compile(loss='categorical_crossentropy',
optimizer= optimizer,
metrics=['accuracy'])


# Reserve 10,000 samples for validation
x_val = x_train_pad[-10000:]
y_val = y_train[-10000:]
x_train = x_train_pad[:-10000]
y_train = y_train[:-10000]

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)
y_val = to_categorical(y_val, 3)

# epoch -> how many times we are going to train our data.
# batch_size -> feeding size
model.fit(x_train, y_train, epochs=50, batch_size=256,validation_data=(x_val, y_val),callbacks=[callback],class_weight=class_weight_dict)


print("Evaluate on test data")
results = model.evaluate(x_test_pad, y_test)
print("test loss, test acc:", results)



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
790/790 [==============================] - 84s 101ms/step - loss: 0.7305 - accuracy: 0.7656 - val_loss: 0.5146 - val_accuracy: 0.7852
Epoch 2/50
790/790 [==============================] - 78s 99ms/step - loss: 0.5922 - accuracy: 0.8217 - val_loss: 0.4458 - val_accuracy: 0.8279
Epoch 3/50
790/790 [==============================] - 78s 99ms/step - loss: 0.5463 - accuracy: 0.8405 - val_loss: 0.4412 - val_accuracy: 0.8309
Epoch 4/50
790/790 [==============================] - 79s 99ms/step - loss: 0.5195 - accuracy: 0.8463 - val_loss: 0.3803 - val_accuracy: 0.8581
Epoch 5/50
790/790 [==============================] - 79s 100ms/step - loss: 0.5030 - accuracy: 0.8516 - val_loss: 0.3509 - val_accuracy: 0.8663
Epoch 6/50
790/790 [==============================] - 79s 100ms/step - loss: 0.4872 - accuracy: 0.8550 - val_loss: 0.3875 - val_accuracy: 0.8499
Epoch 7/50
790/790 [==============================] - 79s 100ms/step - loss: 0.4759 - accuracy: 0.8562 - val_loss: 0.3588 - val_accur

KeyboardInterrupt: ignored